## Kaggle: Sberbank Home price prediction

### Multiple Linear Regression Model

In [1]:
from __future__ import print_function  # Python 2 and 3
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

from sklearn import preprocessing
from sklearn import linear_model

pd.set_option('display.max_columns', None) # to display all columns of the dataframe

%matplotlib inline

In [2]:
# load the train and test datasets

dtrain = pd.read_csv('dtrain_macro_lag6mo.csv', index_col = 0)

dtest = pd.read_csv('dtest_macro_lag6mo.csv', index_col = 0)

In [3]:
dtrain.head()

,timestamp_x,full_sq,life_sq,floor,max_floor,material,num_room,kitch_sq,product_type,sub_area,indust_part,school_education_centers_raion,sport_objects_raion,culture_objects_top_25_raion,oil_chemistry_raion,metro_min_avto,green_zone_km,industrial_km,kremlin_km,radiation_km,ts_km,fitness_km,stadium_km,additional_education_km,cafe_count_1500_price_500,cafe_count_1500_price_high,cafe_count_2000_price_2500,trc_sqm_5000,cafe_count_5000,cafe_count_5000_price_high,park_km,price_doc,log_price_doc,build_year,state,ts_lag6mo,timestamp_y,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_annual,gdp_annual_growth,mortgage_value,mortgage_growth,mortgage_rate,income_per_cap,salary,labor_force,unemployment,employment
0,2011-08-20,43.0,27.0,4.0,12.0,1.0,2.0,6.0,Investment,Bibirevo,0.000070,5,7,0,no,2.590241,0.600973,1.080934,15.156211,1.210027,4.308127,0.485841,8.148591,0.947962,14,0,2,4036616,152,0,2.158587,5850000,15.581952,1966,2,2011-02-20,2011-02-20,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
1,2011-08-23,34.0,19.0,3.0,12.0,1.0,2.0,6.0,Investment,Nagatinskij Zaton,0.049637,8,6,1,no,0.936700,0.065321,0.966479,8.698054,2.724295,0.725560,0.668364,6.127073,1.072315,6,0,2,2034942,177,0,0.550690,6000000,15.607270,1962,2,2011-02-23,2011-02-23,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
2,2011-08-27,43.0,29.0,2.0,12.0,1.0,2.0,6.0,Investment,Tekstil'shhiki,0.118537,7,5,0,no,2.120999,0.453172,0.939275,9.067885,0.772216,3.562188,0.733101,0.767569,0.391957,4,0,1,1572990,122,0,0.374848,5700000,15.555977,1965,2,2011-02-27,2011-02-27,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
3,2011-09-01,89.0,50.0,9.0,12.0,1.0,2.0,6.0,Investment,Mitino,0.069753,10,17,0,no,1.489049,0.106125,0.451173,19.487005,6.348716,0.583025,0.623484,7.385521,0.892674,5,0,1,942180,61,1,0.078090,13100000,16.388123,1965,2,2011-03-01,2011-03-01,111.27,13249.3,5.1,347.9,419.8,46308.5,0.045037,54280,1.344303,12.43,46225.2,44898.7,6643.626,0.014,0.708
4,2011-09-05,77.0,77.0,4.0,12.0,1.0,2.0,6.0,Investment,Basmannoe,0.037316,9,25,0,no,1.257186,0.236214,0.392871,2.578671,1.638318,2.609420,0.220288,3.610754,0.810801,70,1,50,3503058,2283,17,0.258289,16331452,16.608603,1953,2,2011-03-05,2011-03-05,111.27,13249.3,5.1,347.9,419.8,46308.5,0.045037,54280,1.344303,12.43,46225.2,44898.7,6643.626,0.014,0.708


In [4]:
dtest.head()

,timestamp_x,full_sq,life_sq,floor,max_floor,material,num_room,kitch_sq,product_type,sub_area,indust_part,school_education_centers_raion,sport_objects_raion,culture_objects_top_25_raion,oil_chemistry_raion,metro_min_avto,green_zone_km,industrial_km,kremlin_km,radiation_km,ts_km,fitness_km,stadium_km,additional_education_km,cafe_count_1500_price_500,cafe_count_1500_price_high,cafe_count_2000_price_2500,trc_sqm_5000,cafe_count_5000,cafe_count_5000_price_high,park_km,build_year,state,ts_lag6mo,timestamp_y,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_annual,gdp_annual_growth,mortgage_value,mortgage_growth,mortgage_rate,income_per_cap,salary,labor_force,unemployment,employment
0,2015-07-01,39.0,20.7,2,9,1.0,1.0,8.9,Investment,Juzhnoe Butovo,0.041116,13,13,0,no,1.258957,0.061485,1.205404,24.779082,6.620081,8.824060,0.225788,20.392427,0.514211,0,0,0,299166,19,0,2.048487,1998,3,2015-01-01,2015-01-01,46.3414,21514.7,0.4,452.0,513.9,77945.1,0.007065,1753294,0.309669,13.17,39156.0,64310.0,7067.470079,0.017708,0.733598
1,2015-07-01,79.2,32.3,8,17,1.0,3.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,0.007122,0,0,0,no,4.230425,0.000000,0.742377,21.722620,8.545593,3.654955,1.313180,13.445121,1.694967,0,0,1,231300,20,0,4.400427,2015,1,2015-01-01,2015-01-01,46.3414,21514.7,0.4,452.0,513.9,77945.1,0.007065,1753294,0.309669,13.17,39156.0,64310.0,7067.470079,0.017708,0.733598
2,2015-07-01,40.5,25.1,3,5,2.0,2.0,4.8,Investment,Perovo,0.225825,7,13,0,yes,1.585306,0.580638,0.900408,10.320728,0.826743,3.290966,0.966254,1.464174,2.330995,6,0,2,1024431,179,0,2.513006,1960,2,2015-01-01,2015-01-01,46.3414,21514.7,0.4,452.0,513.9,77945.1,0.007065,1753294,0.309669,13.17,39156.0,64310.0,7067.470079,0.017708,0.733598
3,2015-07-01,62.8,36.0,17,17,1.0,2.0,62.8,OwnerOccupier,Poselenie Voskresenskoe,0.017647,0,0,0,no,7.931398,0.025446,0.466738,26.960463,9.434351,10.480798,4.480234,18.560234,0.403429,1,0,0,0,5,0,5.612835,2016,3,2015-01-01,2015-01-01,46.3414,21514.7,0.4,452.0,513.9,77945.1,0.007065,1753294,0.309669,13.17,39156.0,64310.0,7067.470079,0.017708,0.733598
4,2015-07-01,40.0,40.0,17,17,1.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,0.007122,0,0,0,no,2.152792,0.427248,0.353642,21.905792,9.036942,3.729416,1.519553,13.898607,1.339078,0,0,1,201300,20,0,4.566595,2015,1,2015-01-01,2015-01-01,46.3414,21514.7,0.4,452.0,513.9,77945.1,0.007065,1753294,0.309669,13.17,39156.0,64310.0,7067.470079,0.017708,0.733598


#### Remove the timestamp column(s) from both the train and test datasets

In [5]:
dtrain.drop('timestamp_x', axis=1, inplace=True)

In [6]:
dtrain.drop(['ts_lag6mo', 'timestamp_y'], axis=1, inplace=True)

In [7]:
dtest.drop(['timestamp_x', 'ts_lag6mo', 'timestamp_y'], axis=1, inplace=True)

#### Remove outliers (price_doc)

In [8]:
## remove outlier (price_doc) : train dataset
dtrain = dtrain.drop(dtrain[dtrain['price_doc'] > 1e8].index)

In [9]:
dtrain.shape

(30469, 49)

In [10]:
# state : fill state 0 by mode
dtrain.loc[dtrain['state'] == 0, 'state'] = dtrain['state'].mode()[0]
dtest.loc[dtest['state'] == 0, 'state'] = dtest['state'].mode()[0]

In [11]:
# floor: fill missing with mode
dtrain.loc[dtrain['floor'].isnull(), 'floor'] = dtrain['floor'].mode()[0]

In [12]:
dtrain['floor'].isnull().sum()

0

In [13]:
dtrain.head()

,full_sq,life_sq,floor,max_floor,material,num_room,kitch_sq,product_type,sub_area,indust_part,school_education_centers_raion,sport_objects_raion,culture_objects_top_25_raion,oil_chemistry_raion,metro_min_avto,green_zone_km,industrial_km,kremlin_km,radiation_km,ts_km,fitness_km,stadium_km,additional_education_km,cafe_count_1500_price_500,cafe_count_1500_price_high,cafe_count_2000_price_2500,trc_sqm_5000,cafe_count_5000,cafe_count_5000_price_high,park_km,price_doc,log_price_doc,build_year,state,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_annual,gdp_annual_growth,mortgage_value,mortgage_growth,mortgage_rate,income_per_cap,salary,labor_force,unemployment,employment
0,43.0,27.0,4.0,12.0,1.0,2.0,6.0,Investment,Bibirevo,0.000070,5,7,0,no,2.590241,0.600973,1.080934,15.156211,1.210027,4.308127,0.485841,8.148591,0.947962,14,0,2,4036616,152,0,2.158587,5850000,15.581952,1966,2,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
1,34.0,19.0,3.0,12.0,1.0,2.0,6.0,Investment,Nagatinskij Zaton,0.049637,8,6,1,no,0.936700,0.065321,0.966479,8.698054,2.724295,0.725560,0.668364,6.127073,1.072315,6,0,2,2034942,177,0,0.550690,6000000,15.607270,1962,2,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
2,43.0,29.0,2.0,12.0,1.0,2.0,6.0,Investment,Tekstil'shhiki,0.118537,7,5,0,no,2.120999,0.453172,0.939275,9.067885,0.772216,3.562188,0.733101,0.767569,0.391957,4,0,1,1572990,122,0,0.374848,5700000,15.555977,1965,2,100.96,13249.3,5.1,345.2,406.2,46308.5,0.045037,19901,1.482660,12.63,42384.6,44898.7,6643.626,0.014,0.708
3,89.0,50.0,9.0,12.0,1.0,2.0,6.0,Investment,Mitino,0.069753,10,17,0,no,1.489049,0.106125,0.451173,19.487005,6.348716,0.583025,0.623484,7.385521,0.892674,5,0,1,942180,61,1,0.078090,13100000,16.388123,1965,2,111.27,13249.3,5.1,347.9,419.8,46308.5,0.045037,54280,1.344303,12.43,46225.2,44898.7,6643.626,0.014,0.708
4,77.0,77.0,4.0,12.0,1.0,2.0,6.0,Investment,Basmannoe,0.037316,9,25,0,no,1.257186,0.236214,0.392871,2.578671,1.638318,2.609420,0.220288,3.610754,0.810801,70,1,50,3503058,2283,17,0.258289,16331452,16.608603,1953,2,111.27,13249.3,5.1,347.9,419.8,46308.5,0.045037,54280,1.344303,12.43,46225.2,44898.7,6643.626,0.014,0.708


In [14]:
dtrain.isnull().sum()

full_sq                           0
life_sq                           0
floor                             0
max_floor                         0
material                          0
num_room                          0
kitch_sq                          0
product_type                      0
sub_area                          0
indust_part                       0
school_education_centers_raion    0
sport_objects_raion               0
culture_objects_top_25_raion      0
oil_chemistry_raion               0
metro_min_avto                    0
green_zone_km                     0
industrial_km                     0
kremlin_km                        0
radiation_km                      0
ts_km                             0
fitness_km                        0
stadium_km                        0
additional_education_km           0
cafe_count_1500_price_500         0
cafe_count_1500_price_high        0
cafe_count_2000_price_2500        0
trc_sqm_5000                      0
cafe_count_5000             

In [15]:
dtrain['floor'].isnull().sum()

0

#### Convert categorical features to numerical equivalent

In [16]:
# train dataset
for feature in dtrain.columns:
    if dtrain[feature].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(dtrain[feature].values)) 
        dtrain[feature] = lbl.transform(list(dtrain[feature].values))
        

In [17]:
# test dataset
for feature in dtest.columns:
    if dtest[feature].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(dtest[feature].values)) 
        dtest[feature] = lbl.transform(list(dtest[feature].values))


#### Fitting Model

In [21]:
# input variables
X = dtrain[['full_sq','life_sq','floor','max_floor','material','num_room','kitch_sq','product_type','sub_area',
     'indust_part','school_education_centers_raion','sport_objects_raion','culture_objects_top_25_raion',
     'oil_chemistry_raion','metro_min_avto','green_zone_km','industrial_km','kremlin_km','radiation_km',
     'ts_km','fitness_km','stadium_km','additional_education_km','cafe_count_1500_price_500',
     'cafe_count_1500_price_high','cafe_count_2000_price_2500','trc_sqm_5000','cafe_count_5000',
     'cafe_count_5000_price_high','park_km','build_year','state','oil_urals','gdp_quart','gdp_quart_growth',
     'cpi','ppi','gdp_annual','gdp_annual_growth','mortgage_value','mortgage_growth','mortgage_rate','income_per_cap',
     'salary','labor_force','unemployment','employment']]


# Response variable
Y = dtrain['log_price_doc']

ols = linear_model.LinearRegression()
ols.fit(X, Y)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X, Y)))

           


Intercept: 14.971527
Coefficients: [  1.28901243e-02  -4.21550963e-04   6.22303398e-03   1.88286485e-03
  -5.46291955e-03   4.35893264e-02   1.42214101e-03   1.86421098e-01
  -2.59740889e-04  -2.50082600e-01   1.23014439e-02   2.08091458e-04
  -2.98870424e-02  -2.46023460e-02  -1.10301355e-02  -4.30178766e-02
   4.01618977e-02  -1.86812474e-02   1.24475056e-02  -3.98973446e-03
  -3.64445104e-02   2.25164278e-03  -1.64624693e-02  -2.60609824e-03
   3.83977561e-02  -2.91490594e-03   4.76469177e-08   2.78712656e-05
   9.37409250e-03   1.57213450e-02  -8.02531717e-04   7.44749826e-02
   2.02062934e-03  -3.08732621e-06   2.64006524e-02   3.97221149e-03
   4.48068833e-04  -4.58543475e-06   8.01328666e-09   4.85659377e-09
  -1.60539298e-01   4.50673082e-03  -3.45965216e-07   6.75120398e-06
  -8.56691776e-05   6.43422854e-09  -1.13322088e-08]
R^2: 0.374773


#### Variance Inflation Factor (VIF)

In [22]:

from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [26]:
# Running a multiple regression

# input features
features = "+".join(X.columns)

# get y and X dataframes based on this regression
y1, X1 = dmatrices('log_price_doc ~' + features, data = dtrain, return_type='dataframe')


In [29]:
# For each X, calculate VIF and save in dataframe

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif["features"] = X1.columns

In [30]:
vif.round(1)

,VIF Factor,features
0,0.000000,Intercept
1,2.800000,full_sq
2,2.200000,life_sq
3,1.300000,floor
4,1.500000,max_floor
5,1.200000,material
6,1.800000,num_room
7,1.400000,kitch_sq
8,5.500000,product_type
9,1.300000,sub_area


In [34]:
X_new = dtrain[['full_sq','life_sq','floor','max_floor','material','num_room','kitch_sq','product_type','sub_area',
     'indust_part','school_education_centers_raion','sport_objects_raion','culture_objects_top_25_raion',
     'oil_chemistry_raion','metro_min_avto','green_zone_km','industrial_km',
     'fitness_km','additional_education_km',
     'trc_sqm_5000','cafe_count_5000',
     'build_year','state','oil_urals','gdp_quart',
     'ppi','mortgage_value','mortgage_rate','income_per_cap']]

# Response variable
Y_new = dtrain['log_price_doc']

ols = linear_model.LinearRegression()
ols.fit(X_new, Y_new)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X_new, Y_new)))



Intercept: 15.051922
Coefficients: [  1.30244356e-02  -3.59492193e-04   5.82181408e-03   2.52753105e-03
   1.22462955e-03   3.97292282e-02  -1.07179021e-04   1.60491860e-01
  -2.53521416e-04  -2.56670584e-01   1.41329984e-02   5.02128915e-04
  -6.65048244e-02  -2.03062200e-02  -1.00019683e-02  -5.24209328e-02
   6.61543223e-03  -2.77182320e-02  -8.15400140e-03   5.58809900e-08
   1.45832041e-04  -1.19466055e-03   8.05134635e-02   6.02817581e-04
   9.68876799e-06   2.99403074e-03  -1.96218810e-09   1.73160555e-02
   1.22938638e-07]
R^2: 0.363075


In [39]:
X_test = dtest[['full_sq','life_sq','floor','max_floor','material','num_room','kitch_sq','product_type','sub_area',
     'indust_part','school_education_centers_raion','sport_objects_raion','culture_objects_top_25_raion',
     'oil_chemistry_raion','metro_min_avto','green_zone_km','industrial_km',
     'fitness_km','additional_education_km',
     'trc_sqm_5000','cafe_count_5000',
     'build_year','state','oil_urals','gdp_quart',
     'ppi','mortgage_value','mortgage_rate','income_per_cap']]

log_y_pred = ols.predict(X_test)
y_pred = np.exp(log_y_pred) - 1

In [40]:
submit = pd.DataFrame({'id': np.array(dtest.index), 'price_doc': y_pred})
submit.to_csv('submission.csv', index=False)